HW4 Tardigrades.

# 1. Obtaining data 

1.1 Genomic: `wget -o data/Tardi_assebled_genome.fna.gz http://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/949/185/GCA_001949185.1_Rvar_4.0/GCA_001949185.1_Rvar_4.0_genomic.fna.gz` \
1.2 AUGUSTUS results: https://drive.google.com/file/d/1hCEywBlqNzTrIpQsZTVuZk1S9qKzqQAq/view?usp=sharing \
1.3 GFF: https://drive.google.com/file/d/12ShwrgLkvJIYQV2p1UlXklmxSOOxyxj4/view?usp=sharing \
1.4 getAnnoFasta.pl script: http://augustus.gobics.de/binaries/scripts/getAnnoFasta.pl \
1.5 Tandem Mass Spectrometry results: https://disk.yandex.ru/d/xJqQMGX77Xueqg \

# 2. Get number of obtained proteins 

`grep ">" data/augustus.whole.aa | wc -l`
-> 16435 \
2.1 To narrow down the amount of genes we can look for those which take their place in a nucleus, thus having a NLS (Nuclear Localization Signals). Classical NLS (cNLS): This type of signal is recognized by importin α and importin β proteins. The classical NLS is usually a short stretch of basic amino acids, such as lysine or arginine. The consensus sequence for classical NLS is typically (K/R)4-5(K/R)X(K/R).

# 3. Do a local alignment-based search 

3.1 *SP*Option a*SP* \
`conda install -c bioconda blast` \
`makeblastdb -in data/augustus.whole.aa -dbtype prot  -out data/blastp_tardigrade_protein_db` \
`blastp -db data/blastp_tardigrade_protein_db -query data/peptides.fa -outfmt 6 -out data/blastp_tardigrade_filtered_proteins` \
Column names format 6: qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore \
3.2 *SP*Option b*SP* \
`conda install -c bioconda diamond` \
`diamond makedb --in data/augustus.whole.aa  --db data/diamond_tardigrade_protein_db` \
`diamond blastp -d data/diamond_tardigrade_protein_db.dmnd -q data/peptides.fa  -f 6 -o data/diamond_tardigrade_filtered_proteins --very-sensitive` \
Used option b. From TMS created a local database with peptides. Then blasted against fasta file with all proteins.

# 4. Localization prediction 

Get fasta's for filtered genes \
for 3a:
- unique gene names
`awk -F'\t' '{print $2}' data/blastp_tardigrade_filtered_proteins | sort -u | wc -l`
`awk -F'\t' '{print $2}' data/blastp_tardigrade_filtered_proteins | sort -u > data/blastp_unique_genes` \
- get fasta sequences from the original fasta for every unique gene name
`xargs samtools faidx data/augustus.whole.aa < data/blastp_unique_genes > data/blastp_selected_sequences.fasta` \
- get all gene names where prediction is SP
`awk -F'\t' '$2 == "SP" {print $1}' data/blapsp_targetP_output_protein_type > data/blapsp_targetP_selected_genes`
- get sequences for them
`xargs samtools faidx data/augustus.whole.aa < data/blapsp_targetP_selected_genes > data/blapsp_targetP_selected_genes.fasta`

for 3b:
4a. WoLF PSORT  (https://wolfpsort.hgc.jp/) predicts the subcellular localization of proteins. PSORT predicts cellular localization of proteins based on the presence of a signal peptide on their N-terminus. \
Results: \
g4106.t1 details E.R.: 14.5, E.R._golg: 9.5, extr: 7, golg: 3.5, lyso: 3, pero: 2, plas: 1, mito: 1 \
g12510.t1 details plas: 29, cyto: 3 \
(see pdfs) \
4b. TargetP (https://services.healthtech.dtu.dk/service.php?TargetP-2.0) also predicts the subcellular localization of eukaryotic proteins. The location assignment is based on the predicted presence of any of the N-terminal presequences: chloroplast transit peptide (cTP), mitochondrial targeting peptide (mTP) or secretory pathway signal peptide (SP). \
Results: \
(see zip and pdf)
- to filter out only signal peptides from TargetP output and get sequences for them
`awk -F'\t' '$2 ~ /SP/ {print $1}' data/blastp_output_protein_type.tsv > data/blastp_output_protein_type_filtered`
`xargs samtools faidx data/augustus.whole.aa < awk -F'\t' '{print $1}' data/blastp_tardigrade_filtered_proteins > data/blastp_selected_sequences.fasta`

# 5. BLAST

BLAST (https://blast.ncbi.nlm.nih.gov/Blast.cgi) protein sequences against the “UniProtKB/Swiss-Prot” database.
WoLF PSORT:
- g10514.t1 details nucl: 19, cyto_nucl: 15, cyto: 9, extr: 3, mito: 1
- g11320.t1 details plas: 24.5, extr_plas: 16, extr: 6.5, lyso: 1 *SP*
- g11513.t1 details cyto: 17, cyto_nucl: 12.8333, cyto_mito: 9.83333, nucl: 7.5, E.R.: 3, mito: 1.5, plas: 1, pero: 1, golg: 1
- g11806.t1 details nucl: 18, cyto_nucl: 11.8333, mito: 5, extr: 4, cyto: 3.5, cyto_pero: 2.66667, cysk_plas: 1
- g11960.t1 details nucl: 32
- g12388.t1 details extr: 25, plas: 4, mito: 2, lyso: 1 *SP*
- g12510.t1 details plas: 29, cyto: 3
- g12562.t1 details extr: 30, lyso: 2 *SP*
- g1285.t1 details extr: 25, plas: 5, mito: 1, lyso: 1 *SP*
- g13530.t1 details extr: 13, nucl: 6.5, lyso: 5, cyto_nucl: 4.5, plas: 3, E.R.: 3, cyto: 1.5 *SP*
- g14472.t1 details nucl: 28, plas: 2, cyto: 1, cysk: 1
- g15153.t1 details extr: 32 *SP*
- g15484.t1 details nucl: 17.5, cyto_nucl: 15.3333, cyto: 12, cyto_mito: 6.83333, plas: 1, golg: 1
- g16318.t1 details nucl: 20.5, cyto_nucl: 13, extr: 5, cyto: 4.5, E.R.: 1, golg: 1
- g16368.t1 details nucl: 20.5, cyto_nucl: 13, extr: 5, cyto: 4.5, E.R.: 1, golg: 1
- g2203.t1 details plas: 29, nucl: 2, golg: 1
- g3428.t1 details mito: 18, cyto: 11, extr: 2, nucl: 1
- g3679.t1 details extr: 26, mito: 2, lyso: 2, plas: 1, E.R.: 1 *SP*
- g4106.t1 details E.R.: 14.5, E.R._golg: 9.5, extr: 7, golg: 3.5, lyso: 3, pero: 2, plas: 1, mito: 1
- g4970.t1 details plas: 32
- g5237.t1 details plas: 24, mito: 8
- g5443.t1 details extr: 28, nucl: 3, cyto: 1
- g5467.t1 details extr: 27, plas: 4, mito: 1 *SP*
- g5502.t1 details extr: 31, lyso: 1 *SP*
- g5503.t1 details extr: 29, plas: 1, mito: 1, lyso: 1 *SP*
- g5510.t1 details plas: 23, mito: 7, E.R.: 1, golg: 1
- g5616.t1 details extr: 31, mito: 1 *SP*
- g5641.t1 details extr: 31, lyso: 1 *SP*
- g5927.t1 details nucl: 30.5, cyto_nucl: 16.5, cyto: 1.5
- g702.t1 details extr: 29, plas: 2, lyso: 1 *SP*
- g7861.t1 details nucl: 16, cyto_nucl: 14, cyto: 8, plas: 5, pero: 1, cysk: 1, golg: 1
- g8100.t1 details nucl: 16.5, cyto_nucl: 12.5, cyto: 7.5, plas: 5, extr: 2, E.R.: 1
- g8312.t1 details nucl: 15.5, cyto_nucl: 15.5, cyto: 12.5, mito: 2, plas: 1, golg: 1